In [809]:
import requests
import json
import subprocess
import pandas as pd
import datetime
import logging

In [810]:
def city_lon_lat(row):
    city_name=row['City name']
    country_code=row['Country Abbrev.']
    global openweathermap_api_key
    url = f'http://api.openweathermap.org/geo/1.0/direct?q={city_name},{country_code}&limit=5&appid={openweathermap_api_key}'
    response = requests.get(url)

    if response.status_code == 200:
        forecast_data_of_city = response.json()  # Convert response to JSON format
        #print(forecast_data_of_city)
    else:
        raise Exception('Error with openweathermap:', response.status_code, response.reason)

    lat=forecast_data_of_city[0]['lat']
    lon=forecast_data_of_city[0]['lon']
    return({'lat':lat,'lon':lon})


In [811]:
# #api call to get CURRENT weather:
# url = f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&units=metric&appid={openweathermap_api_key}'
# response = requests.get(url)

# if response.status_code == 200:
#     current_weather_data = response.json()  # Convert response to JSON format
#     (json.dumps(current_weather_data,indent=4))
#     # Process the data as needed
# else:
#     print('Error:', response.status_code )
#     print('parameters:',response)

# '''extract the:
# 1. weather description
# 2. icon
# 3. clouds%
# 4.sunset + sunrise
# 5. city name
# '''
# type(current_weather_data)
# pd.json_normalize(current_weather_data)
# for i in current_weather_data.items():
#     print(i)

# # Convert Unix timestamp to UTC datetime object
# sunrise_gmt_plus_2_datetime  = datetime.datetime.utcfromtimestamp(current_weather_data['sys']['sunrise'])+ datetime.timedelta(hours=2)
# sunset_gmt_plus_2_datetime = datetime.datetime.utcfromtimestamp(current_weather_data['sys']['sunset']) + datetime.timedelta(hours=2)

# # Print the human-readable date and time in GMT+2
# print(f"sunset is at: {sunset_gmt_plus_2_datetime.strftime('%Y-%m-%d %H:%M:%S')}",
#       f"sunrise is at: {sunrise_gmt_plus_2_datetime.strftime('%Y-%m-%d %H:%M:%S')}" )


In [812]:
   
def convert_to_gmt_plus_2(timestamp):
        utc_datetime = datetime.datetime.utcfromtimestamp(timestamp)
        gmt_plus_2_datetime = utc_datetime + datetime.timedelta(hours=2)
        return gmt_plus_2_datetime


In [813]:
def weather_forecast_for_city(openweathermap_api_key,city_name,lat,lon):
    # api to get forecast weather for the next 5 days, every 3 hours:
    url = f'https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&units=metric&appid={openweathermap_api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        weather_data = response.json()  # Convert response to JSON formatd
    else:
        raise Exception('Error with api.openweathermap:', response.status_code,response)
    df_with_weather_col_as_list=pd.json_normalize(weather_data['list'])
    df_with_weather_col_as_list['city_name']=city_name
    # since weather column is list of dict (which json_normalize couldnt handle) I had to concat it to the full table, with a prefix or 'weather.' for each column.
    df_with_weather_col_normalized=pd.concat([df_with_weather_col_as_list.drop(['weather'],axis=1),pd.json_normalize(weather_data['list'],'weather',record_prefix='weather.')],axis=1)
    df_weather_prepared= df_with_weather_col_normalized #df_with_weather_col_normalized[interesting_columns]
    df_weather_prepared.loc[:,'Forecast Time (Israel Time)']=df_weather_prepared['dt'].apply(convert_to_gmt_plus_2)
    return df_weather_prepared

In [814]:
# import matplotlib.pyplot as plt

# forecast_data=df
# # Assuming df is your DataFrame containing weather forecast data
# # Convert 'Forecast Time (Israel Time)' to datetime
# forecast_data.loc[:,'Forecast Time (Israel Time)'] = pd.to_datetime(forecast_data['Forecast Time (Israel Time)'])
# fig,ax1=plt.subplots()
# # Plot main.feels_like (temperature) as a line
# forecast_data_day=forecast_data[forecast_data['sys.pod']=='d']
# forecast_data_night=forecast_data[forecast_data['sys.pod']=='n']
# ax1.plot(forecast_data_day['Forecast Time (Israel Time)'], forecast_data_day['main.feels_like'], color='orange', label='Day Temperature')
# ax1.plot(forecast_data_night['Forecast Time (Israel Time)'], forecast_data_night['main.feels_like'], color='black', label='Night Temperature')
# ax1.set_ylabel('Temparture(Celsius)')
# light_blue_color = (173 / 255, 216 / 255, 230 / 255)
# ax2=ax1.twinx()
# ax2.plot(forecast_data_day['Forecast Time (Israel Time)'], forecast_data_day['clouds.all']/100,color=light_blue_color,label='Day Cloudiness %')
# ax2.set_ylabel('Cloudiness (%)')

# # Combine legends
# lines1, labels1 = ax1.get_legend_handles_labels()
# lines2, labels2 = ax2.get_legend_handles_labels()
# lines = lines1 + lines2
# labels = labels1 + labels2
# ax1.legend(lines, labels, loc='upper left')
# plt.show()


# #another chart:
# light_blue_color = (255/ 255, 165/ 255, 0/ 255)  # RGB: (255, 165, 0)
# dark_blue_color =  (0, 0, 0/ 255)       # RGB: (0, 0, 0)
# plt.bar(forecast_data_day['Forecast Time (Israel Time)'], forecast_data_day['rain.3h'], color=[light_blue_color])
# plt.bar(forecast_data_night['Forecast Time (Israel Time)'], forecast_data_night['rain.3h'], color=[dark_blue_color],alpha=0.3)
# plt.xticks( rotation=45)
# plt.xlabel('Date',loc='center')
# plt.show()

# #another chart:
# plt.plot(forecast_data_day['Forecast Time (Israel Time)'], forecast_data_day['wind.speed'], color='orange', label='Temperature')
# plt.plot(forecast_data_night['Forecast Time (Israel Time)'], forecast_data_night['wind.speed'], color='black', label='Temperature')
# plt.xlabel('Date',loc='center')
# plt.xticks( rotation=45)
# plt.show()





In [815]:
def get_air_pollution_city(openweathermap_api_key,city_name,lat,lon):
    __doc__=\
    '''
    bring the air pollution forecast for the next 4 days with hourly granularity
    '''
    def aqi_desc(aqi):
        match aqi:
            case 1:return 'Good'
            case 2:return 'Fair'
            case 3:return 'Moderate'
            case 4:return 'Poor'
            case 5:return 'Very Poor'
            case _:return 'N/A'  # Default case for values not covered in the predefined cases
    
    ## API for Air Pollution:
    air_pollution_url=f'http://api.openweathermap.org/data/2.5/air_pollution/forecast?lat={lat}&lon={lon}&appid={openweathermap_api_key}'
    response=requests.get(air_pollution_url)
    if response.status_code==200:
        air_pol_dict_type=response.json()
    else:
        raise Exception(response.status_code)
    df_air_pollution=pd.json_normalize(air_pol_dict_type['list'])
    df_air_pollution_subset=df_air_pollution[['dt','main.aqi']].copy()
    df_air_pollution_subset['aqi desc']=df_air_pollution_subset['main.aqi'].apply(aqi_desc)
    df_air_pollution_subset['dt GMT']=df_air_pollution_subset['dt'].apply(convert_to_gmt_plus_2)
    df_air_pollution_subset['city_name']=city_name
    return(df_air_pollution_subset)

    #with open('air_pollution_forecast.csv','w') as f: 
        #df_air_pollution.to_csv(f,index=False,lineterminator='\n')
    

In [816]:
def get_token_from_amadeus():
    client_id = 'xNoWUqmFVVeDmNjPAQAal3w9CBngCr0N'
    client_secret = 'c8U8GSe7CANowHZQ'

    # Define the command and its arguments as a list
    command = [
        'curl',
        '--insecure',
        'https://test.api.amadeus.com/v1/security/oauth2/token',
        '-H',
        'Content-Type: application/x-www-form-urlencoded',
        '-d',
        f'grant_type=client_credentials&client_id={client_id}&client_secret={client_secret}'
    ]

    # Run the command
    try:
        output = subprocess.run(command, capture_output=True, text=True)
    except:
        raise Exception('Error occured in api.amadeus: ',Exception)         
    else: 
        amadeus_access_token=json.loads(output.stdout)['access_token']
        print('Token recieved successfully from Amadeus')
        print('please wait while creating weather forecast for all avaliable flight destinations...')
        return(amadeus_access_token)

    
    
    


In [817]:
def get_possible_flight_destinations(amadeus_access_token,origin,departureDate,oneWay,duration,nonStop,maxPrice,viewBy):
    command=[
    'curl',
    '--insecure',
    f'https://test.api.amadeus.com/v1/shopping/flight-destinations?origin={origin}&departureDate={departureDate}&oneWay={oneWay}&duration={duration}&nonStop={nonStop}&maxPrice={maxPrice}&viewBy={viewBy}',
    '-H',
    f'Authorization: Bearer {amadeus_access_token}'
    ]
    # Run the command
    try:
        output = subprocess.run(command, capture_output=True, text=True)
    except Exception:
        raise Exception('Error Occured with getting possible flights from Amadeus:',Exception)
    else:
        try:
            flight_destinations=pd.json_normalize(json.loads(output.stdout)['data'])
        except KeyError:
             print('Error occured when trying to get flight destinations: ',json.loads(output.stdout)['errors'][0]['detail'])
             logging.exception(Exception)
             raise Exception(KeyError)
        except Exception:
             print ('the error: ' ,Exception)
             logging.exception(Exception)
             exit(1)
        else: 
            try:
                with open('flight_destinations.csv','w') as f:
                        flight_destinations.to_csv(f,lineterminator='\n',index=False,header=True,sep=',')
            except FileExistsError:
                raise Exception('file flight_destinations.csv already exists:' ,FileExistsError)
            except PermissionError:
                raise Exception('File is flight_destinations.csv is open. please close it')
            except Exception as e:
                    logging.exception(e)
                    raise Exception(Exception)
            return(flight_destinations)

In [818]:
def get_cities_names_with_flight_destinations(flight_destinations):
    #hard copy csv file for aiport-codes has been downloaded in advance:
    try:
        airport_codes=pd.read_csv('airport-codes.csv', encoding='ISO-8859-1')
    except FileNotFoundError: 
            logging.exception(Exception)
            raise Exception("File airport-codes.csv not found")
    except Exception:
         raise Exception('inside function: get_cities_names_with_flight_destinations',Exception)
    else:
        airport_codes_fixed=airport_codes.iloc[:,[0,1,3,4]].dropna(subset=['City name','Airport Code','Country name'])
        with open('airport_codes_fixed.csv','w') as f:
            airport_codes_fixed.to_csv(f,lineterminator='\n',header=True,index=False)
        df_airport_codes_cities=pd.merge(left=flight_destinations[['destination']],\
                right=airport_codes_fixed,\
                left_on='destination',\
                right_on='Airport Code',\
                how='inner')
        df_airport_codes_cities['city_name']=df_airport_codes_cities['City name'].str.replace(' ','-')
        return df_airport_codes_cities

In [819]:
def is_date_relvant(row,flight_destinations,df_airport_codes_cities):

    flight_destinations_with_city_names=pd.merge(left=flight_destinations,\
                                                 right=df_airport_codes_cities,\
                                                 left_on='destination',\
                                                 right_on='destination',\
                                                 how='left')
    with open('flight_destinations_with_city_names.csv','w')as f:
        flight_destinations_with_city_names.to_csv(f,lineterminator='\n',header=True,index=False)
    dt = row['dt GMT']  # Assuming 'dt GMT' is the column containing Timestamp objects
    city_name = row['city_name']
    # Convert 'departureDate' and 'returnDate' columns to datetime-like values
    #print(flight_destinations_with_city_names)
    #print(flight_destinations_with_city_names.describe())
    #print(flight_destinations_with_city_names.info())
    flight_destinations_with_city_names['departureDate'] = pd.to_datetime(flight_destinations_with_city_names['departureDate'])
    flight_destinations_with_city_names['returnDate'] =  pd.to_datetime(flight_destinations_with_city_names['returnDate'])
    #print(flight_destinations_with_city_names)
    relevant_dates = flight_destinations_with_city_names[(flight_destinations_with_city_names['departureDate'].dt.date <= dt.date()) & \
                                                        (flight_destinations_with_city_names['returnDate'].dt.date >= dt.date()) & \
                                                        (flight_destinations_with_city_names['city_name'] == city_name)]
    
    return 1 if not relevant_dates.empty else 0



In [820]:
def main():
    try:

        openweathermap_api_key='1d1e6d6d2d0af039a64927a9c8578fad'
        amadeus_access_token=get_token_from_amadeus()
        flight_destinations=get_possible_flight_destinations(amadeus_access_token,origin='TLV',departureDate='2024-03-19',oneWay='false',duration='3,10',nonStop='false',maxPrice=1200,viewBy='DATE')
        df_airport_codes_cities=get_cities_names_with_flight_destinations(flight_destinations)
        df_airport_codes_cities['lon_lan']=df_airport_codes_cities.apply(city_lon_lat,axis=1)
        cities_dict=(df_airport_codes_cities[['city_name','lon_lan']]).to_dict(orient='tight')
        weather_forecast_for_all_destinations=pd.DataFrame()
        air_pollution_for_all_destinations=pd.DataFrame()
        #looping on all city destinations and extracting the weather and air-pollution forecast, into a one big data frame:
        for i in range(len(cities_dict['data'])):
            city_name=cities_dict['data'][i][0]
            lat=cities_dict['data'][i][1]['lat']
            lon=cities_dict['data'][i][1]['lon']
            weather_forecast_for_all_destinations=pd.concat([weather_forecast_for_all_destinations,weather_forecast_for_city(openweathermap_api_key,city_name,lat,lon)])
            air_pollution_for_all_destinations=pd.concat([air_pollution_for_all_destinations,get_air_pollution_city(openweathermap_api_key,city_name,lat,lon)])   
            #print('city name: ',city_name,'lat: ',lat,'lon: ',lon)
        weather_air_pol_for_all_destinations=pd.merge(left=weather_forecast_for_all_destinations,\
                                                    right=air_pollution_for_all_destinations,\
                                                    how='left',\
                                                    left_on=['dt','city_name'],\
                                                    right_on=['dt','city_name'])    
        weather_air_pol_for_all_destinations['relevant_for_flight'] = weather_air_pol_for_all_destinations.apply(is_date_relvant,args=[flight_destinations,df_airport_codes_cities], axis=1)

        relvant_columns=['relevant_for_flight','city_name','pop','dt','dt_txt','Forecast Time (Israel Time)','aqi desc',\
                         'visibility','rain.3h','main.feels_like','main.temp_min','main.temp_max','clouds.all','wind.speed',\
                            'wind.deg','sys.pod','weather.main','weather.description','weather.icon']
        try:
            with open('weather_air_pol_for_all_destinations.csv','w') as f:              
                weather_air_pol_for_all_destinations[relvant_columns].to_csv(f,sep=',',lineterminator='\n',index=False)
        except PermissionError:
            raise Exception('File weather_air_pol_for_all_destinations.csv is open. please close it')
        except Exception as e:
            raise Exception(e)     
    except Exception as e:
        print ('An error occured: ',e)
        raise Exception(e)
        
if __name__ == "__main__":
    main()

Token recieved successfully from Amadeus
